# **Test 080424**

In [137]:
import pandas as pd
import json
import csv
import os

In [138]:
def read_data(file_path):
    """
       params:
           file_path: 文件路径
       return:
           data: 读取的数据列表，每行一条样本

    """
    data = []
    with open(file_path, "r") as f:
        for line in f.readlines():
            # 过长的评论文本会超出io限制报错。 暂时忽略
            try:\
                # 字符串预处理
                # 布尔替换成python的习惯
                str_text = line.replace("true", "True")
                str_text = str_text.replace("false", "False")
                # 转成字典形式
                raw_sample = json.loads(str_text)
                print(raw_sample)
                data.append(raw_sample)
            except:
                print(line)
                pass
    return data

def read_and_split_data(file_path, batch_size=10000):
    """
       params:
           file_path: 文件路径
       return:
           data: 读取的数据列表，每行一条样本

    """
    data = []
    small_list_data = []
    count = 0
    with open(file_path, "r") as f:
        for line in f.readlines():
            if count < batch_size:
                # 过长的评论文本会超出io限制报错。 暂时忽略
                try:
                    # 字符串预处理
                    # 布尔替换成python的习惯
                    str_text = line.replace("true", "True")
                    str_text = str_text.replace("false", "False")
                    # 转成字典形式
                    raw_sample = eval(str_text)
                    data.append(raw_sample)
                    count += 1
                except:
                    pass
            else:
                small_list_data.append(data)
                data = []
                count = 0
    small_list_data.append(data)
    return small_list_data

def read_txt_file_to_list(file_path):
    records = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            records.append(line.strip())
    return records


def remove_duplicate_rows(df):
    # Loại bỏ các hàng trùng lặp dựa trên cột 'reviewText'
    df = df.drop_duplicates(subset=['reviewText', 'asin', 'reviewerID'])
    return df

def remove_duplicate_rows_ver2(df):
    # Loại bỏ các hàng trùng lặp dựa trên cột 'reviewText'
    df = df.drop_duplicates(subset=['reviewText', 'itemID', 'reviewerID'])
    return df

In [144]:
file_number = 0
dataset_name = "Digital_Music_5"
json_file_path = "./DeepCGSR_triet/data/raw/Digital_Music_5.json"  # Thay đổi đường dẫn tới file JSON của bạn
output_directory = "./DeepCGSR_triet/data/DigitalMusic/"  # Thay đổi đường dẫn thư mục lưu file CSV của bạn
small_dataset = "Small_Digital_Music_10000_" + str(file_number) + ".json"
chunk_size = 10000  # Kích thước của mỗi phần dữ liệu

extracted_data = output_directory + "extract/" + dataset_name + "_extracted_data_" + str(file_number) + ".csv"
filtered_data = output_directory + "filtered/" + dataset_name + "_filtered_data_" + str(file_number) + ".csv"
outliner_data = output_directory + "outliner/" + dataset_name + "_outliner_data_" + str(file_number) + ".csv"


In [145]:


small_list_data = read_and_split_data(json_file_path, chunk_size)
count_name = 0
for data in small_list_data:
    # Chuyển đổi dữ liệu thành DataFrame
    data_df = pd.DataFrame(data)
    print(data_df)
    # data_df.columns = ['reviewerID', 'asin', 'overall', 'reviewText']

    # Lưu dữ liệu dưới dạng CSV
    output_file_path = os.path.join(output_directory, f"Small_Digital_Music_{len(data_df)}_{count_name}.json")
    data_df.to_json(output_file_path, orient='records', lines=True)

    print(f"Saved {len(data_df)} records to {output_file_path}")
    count_name += 1

      overall vote  verified   reviewTime      reviewerID        asin  \
0         5.0    3      True   06 3, 2013  A2TYZ821XXK2YZ  3426958910   
1         5.0  NaN      True  10 11, 2014  A3OFSREZADFUDY  3426958910   
2         5.0  NaN      True  02 11, 2014  A2VAMODP8M77NG  3426958910   
3         4.0    3     False   12 7, 2013   AAKSLZ9IDTEH0  3426958910   
4         5.0  NaN      True  06 12, 2016  A3OH43OZJLKI09  5557706259   
...       ...  ...       ...          ...             ...         ...   
9995      5.0  NaN      True   01 4, 2015  A1KBHQWJYKYA01  B000WOXPNS   
9996      5.0  NaN      True  11 28, 2014  A1KRWR5D81AXJE  B000WOXPNS   
9997      5.0  NaN      True   11 4, 2014  A2G6VF7XDPKFG3  B000WOXPNS   
9998      5.0  NaN      True  07 30, 2014  A3QXAIQ3EHY7AD  B000WOXPNS   
9999      5.0  NaN      True  06 16, 2014  A13ZE9XWRQPPYC  B000WOXPNS   

                          style       reviewerName  \
0      {'Format:': ' Audio CD'}            Garrett   
1      {'Format

In [146]:
data = pd.read_json(output_directory + small_dataset, lines=True)
print(data)
data_df = pd.DataFrame(data, columns=['reviewerID', 'asin', 'overall', 'reviewText'])
print(data_df.head())
data_df.columns = ['reviewerID', 'asin', 'overall', 'reviewText']

data_rating = data_df["overall"].tolist()
data_review = data_df["reviewText"].tolist()
data_reviewerID = data_df["reviewerID"].tolist()
data_itemID = data_df["asin"].tolist()

      overall  vote  verified   reviewTime      reviewerID        asin  \
0           5   3.0      True   06 3, 2013  A2TYZ821XXK2YZ  3426958910   
1           5   NaN      True  10 11, 2014  A3OFSREZADFUDY  3426958910   
2           5   NaN      True  02 11, 2014  A2VAMODP8M77NG  3426958910   
3           4   3.0     False   12 7, 2013   AAKSLZ9IDTEH0  3426958910   
4           5   NaN      True  06 12, 2016  A3OH43OZJLKI09  5557706259   
...       ...   ...       ...          ...             ...         ...   
9995        5   NaN      True   01 4, 2015  A1KBHQWJYKYA01  B000WOXPNS   
9996        5   NaN      True  11 28, 2014  A1KRWR5D81AXJE  B000WOXPNS   
9997        5   NaN      True   11 4, 2014  A2G6VF7XDPKFG3  B000WOXPNS   
9998        5   NaN      True  07 30, 2014  A3QXAIQ3EHY7AD  B000WOXPNS   
9999        5   NaN      True  06 16, 2014  A13ZE9XWRQPPYC  B000WOXPNS   

                          style       reviewerName  \
0      {'Format:': ' Audio CD'}            Garrett   
1  

In [147]:
print(len(data_rating))
print(len(data_reviewerID))

10000
10000


In [148]:
import pandas as pd
# step01:
def defineRating(data_rating):
  T_v = []
  for rating in data_rating:
    if rating >= 4:
      T_v.append(1)
    else:
      T_v.append(-1)
  return T_v

T_v = defineRating(data_rating)

# Define the data for the dataframe
data = {'id': range(len(T_v)), 'T_v': T_v}

# Create the dataframe
df_rating = pd.DataFrame(data)

# Print the dataframe
print(df_rating)


        id  T_v
0        0    1
1        1    1
2        2    1
3        3    1
4        4    1
...    ...  ...
9995  9995    1
9996  9996    1
9997  9997    1
9998  9998    1
9999  9999    1

[10000 rows x 2 columns]


In [149]:
# Thêm vào dataframe df_rating cột C_v
df_rating['reviewerID'] = data_reviewerID
df_rating['asin'] = data_itemID
df_rating['overall'] = data_rating
df_rating['reviewText'] = data_review
# Loại bỏ các dòng có reviewText trùng lặp
# df_rating = remove_duplicate_rows(df_rating)
print(df_rating)

        id  T_v      reviewerID        asin  overall  \
0        0    1  A2TYZ821XXK2YZ  3426958910        5   
1        1    1  A3OFSREZADFUDY  3426958910        5   
2        2    1  A2VAMODP8M77NG  3426958910        5   
3        3    1   AAKSLZ9IDTEH0  3426958910        4   
4        4    1  A3OH43OZJLKI09  5557706259        5   
...    ...  ...             ...         ...      ...   
9995  9995    1  A1KBHQWJYKYA01  B000WOXPNS        5   
9996  9996    1  A1KRWR5D81AXJE  B000WOXPNS        5   
9997  9997    1  A2G6VF7XDPKFG3  B000WOXPNS        5   
9998  9998    1  A3QXAIQ3EHY7AD  B000WOXPNS        5   
9999  9999    1  A13ZE9XWRQPPYC  B000WOXPNS        5   

                                             reviewText  
0     This is awesome to listen to, A must-have for ...  
1                                                  bien  
2     It was great to hear the old stuff again and I...  
3     well best of's are a bit poison normally but t...  
4     What can I say? This is Casting

# SODCM

In [84]:
from textblob import TextBlob


# Define function to extract a string's sentiment
find_sentiment = lambda text: TextBlob(text).sentiment[0]
# Create new column in dataframe
C_v = data_df["reviewText"].apply(find_sentiment)



TypeError: The `text` argument passed to `__init__(text)` must be a string, not <class 'NoneType'>

In [ ]:
# Thêm vào dataframe df_rating cột C_v
df_rating['C_v'] = C_v
print(df_rating[df_rating['id'] == 98])

In [ ]:
# step02:
import math


def euclidean_distance(p1, p2):
  distance = math.sqrt((p2 - p1)**2)
  return distance
V_d = []
for ti, ci in zip(T_v, C_v):
  V_d.append(euclidean_distance(ti, ci))
  # V_d.append((ti + ci)/2)

In [ ]:
# Thêm vào dataframe df_rating cột V_d
df_rating['V_d'] = V_d
print(df_rating)

In [ ]:
# step03:
S_pos = []
S_neg = []

i = 0
for r in data_rating:
  if r >= 4:
    S_pos.append([i, r, V_d[i]])
  else:
    S_neg.append([i, r, V_d[i]])
  i = i + 1

print(len(S_pos))
print(len(S_neg))
element = None
i = 0
for item in S_pos:
    if item[0] == 25:
        element = item
        break
    i = i + 1
# for item in S_neg:
#     if item[0] == 25:
#         element = item
#         break
print(S_pos[17])
print(element)



In [ ]:
#step04:

def calculation_IQR(S, V_d, flag='pos'):
  # Sắp xếp dữ liệu
  sorted_data = sorted(V_d)

  # Tính Q1 và Q3
  q1_index = int(len(sorted_data) * 0.25)
  q3_index = int(len(sorted_data) * 0.75)
  Q1 = sorted_data[q1_index]
  Q3 = sorted_data[q3_index]

  # Tính IQR
  IQR = Q3 - Q1

  # Tính F_L và F_U
  F_L = Q1 - 1.5 * IQR
  F_U = Q3 + 1.5 * IQR

  if flag == 'pos':
    if F_L < 0:
      O_s =  Q3 + IQR
    else:
      O_s = F_U - IQR*F_L
  else:
    if F_U > 2:
      O_s = Q1 - IQR
    else:
      O_s = Q3 - IQR*F_U

  return O_s

O_Spos = calculation_IQR(S_pos, V_d, flag = 'pos')
O_Sneg = calculation_IQR(S_neg, V_d, flag = 'neg')

print(O_Spos)
print(O_Sneg)

In [ ]:
#step05:


# S_pos: danh sách các chỉ số của các phần tử thuộc S+
# S_neg: danh sách các chỉ số của các phần tử thuộc S-
# VD: danh sách các giá trị của biến VD
# OS_pos: giá trị OS+ như trong mã script
# OS_neg: giá trị OS- như trong mã script

# Khởi tạo danh sách O+ và O-
O_pos = [0] * len(S_pos)
O_neg = [0] * len(S_neg)

# print(S_pos)
# print(S_neg)
count = 0
# Điều chỉnh danh sách O+ và O- theo điều kiện trong mã script
for i, ri in enumerate(S_pos):
  print(i, ri)
  if ri[2] >= O_Spos:
    O_pos[i] = 'yes'
    count += 1
  else:
    O_pos[i] = 'no'

for i, ri in enumerate(S_neg):
    if ri[2] <= O_Sneg:
      O_neg[i] = 'yes'
      count += 1
    else:
      O_neg[i] = 'no'

print(count)
# In ra kết quả
# print("O+:", O_pos)
# print("O-:", O_neg)


In [ ]:
df = pd.DataFrame(S_pos, columns=['id', 'rating', 'V_d'])
df['O_pos'] = O_pos
#In ra các phần tử có giá trị 'yes'
print(df[df['O_pos'] == 'yes'])


In [ ]:
#step06:
# Hàm toggle để đảo ngược giá trị của một biến
def toggle(value):
  if value == -1:
    return 1
  else:
    return -1

# Khởi tạo danh sách TV với giá trị False cho mỗi phần tử
TV_new = T_v.copy()

count_1 = 0
for i, ri in enumerate(S_pos):
    if O_pos[i] == 'yes':
      count_1 += 1
      TV_new[ri[0]] = toggle(T_v[ri[0]])

for i, ri in enumerate(S_neg):
    if O_neg[i] == 'no':
      count_1 += 1
      TV_new[ri[0]] = toggle(T_v[ri[0]])

# print(TV_new)
# Tạo danh sách D* bằng cách nối S+ và S-
print(count_1)
# D_star = S_pos + S_neg

In [ ]:
# Thêm vào dataframe df_rating cột TV_new
df_rating['TV_new'] = TV_new
print(df_rating)

In [ ]:
# Tạo DataFrame mới để lưu các hàng thỏa mãn điều kiện
outliner = pd.DataFrame(columns=df_rating.columns)

# Lặp qua các hàng của DataFrame df_rating
for index, row in df_rating.iterrows():
    if row['TV_new'] != row['T_v']:
        # Thêm hàng vào DataFrame outliner
        outliner.loc[len(outliner)] = row

# In ra DataFrame outliner
print(outliner)

# Lưu DataFrame outliner vào file CSV
outliner.to_csv('/home/triet/ComputerScience_K32/Recommendation_System/DeepCGSR_triet/outliner.csv', index=False)

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax
import torch

# Load pre-trained model and tokenizer for Amazon reviews
model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'  # Example model, you can choose other pre-trained models
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)




In [ ]:
#Define the text to analyze
text = "I love the product"
def sentiment_analysis(text, model, tokenizer):
    # Tokenize the text
    inputs = tokenizer(text, return_tensors='pt')

    # Perform sentiment analysis
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted probabilities for each class
    probs = softmax(outputs.logits, dim=1).squeeze()

    # Convert the probabilities to human-readable labels
    labels = [1, 2, 3, 4, 5]  # For this model, it predicts star ratings
    predicted_label = labels[torch.argmax(probs).item()]
    # print(predicted_label)
    return predicted_label

# Print the result
print("Predicted sentiment:", sentiment_analysis(text, model, tokenizer))
print("Predicted sentiment:", torch.sigmoid(outputs.logits).squeeze()[1].item())
# print("Confidence:", probs[torch.argmax(probs)].item())

In [ ]:

predict_rating = df_rating['reviewText'].apply(lambda x: sentiment_analysis(x[:512], model=model, tokenizer=tokenizer))

predict_rating_list = predict_rating.tolist()
predict_rating_list =  defineRating(predict_rating_list)
print(predict_rating_list)

df_rating['predict_rating'] = predict_rating_list
print(df_rating)


In [ ]:
print(sentiment_analysis("The color pattern and fit is what I liked the most what I liked the least is that they are not easy to clean and stains do not come out very easy or at all", model, tokenizer))

In [ ]:
i = 0

for row in df_rating.itertuples():
    if row.predict_rating != row.T_v:
        i += 1
        print(row.predict_rating)
        print(row.T_v)
        print(row.overall)
        print(row.reviewText)
        print('------------------------')
print('count: ', i)

# Tạo DataFrame mới để lưu các hàng thỏa mãn điều kiện
outliner = pd.DataFrame(columns=['predict_rating', 'T_v', 'overall', 'reviewText'])
# Lặp qua các hàng của DataFrame df_rating
# Lặp qua các hàng của DataFrame df_rating
for index, row in df_rating.iterrows():
    if row['predict_rating'] != row['T_v']:
        # Thêm hàng vào DataFrame outliner
        outliner.loc[len(outliner)] = row

# In ra DataFrame outliner
print(outliner)

# Lưu DataFrame outliner vào file CSV
outliner.to_csv('/home/triet/ComputerScience_K32/Recommendation_System/DeepCGSR_triet/outliner_bert.csv', index=False)

# BERT

In [150]:
import nltk
import math
import spacy
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from flair.data import Sentence
from flair.nn import Classifier
from flair.models import TextClassifier

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

pos_words = read_txt_file_to_list("./DeepCGSR_triet/data/positive-words.txt")
neg_words = read_txt_file_to_list("./DeepCGSR_triet/data/negative-words.txt")
print(pos_words)
print(neg_words)
a = ['JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS']
brands = ['dove', 'aveda', 'silver']
keywords = ['overall', 'summary']

tagger = Classifier.load('pos')
# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")
def remove_stopwords(sentence):
    """
    Loại bỏ các từ dừng từ câu sử dụng NLTK.

    Parameters:
    - sentence (str): Câu cần loại bỏ từ dừng.

    Returns:
    - str: Câu sau khi loại bỏ các từ dừng.
    """
    # Tách câu thành các từ
    words = word_tokenize(sentence)

    # Tải danh sách từ dừng từ NLTK
    stop_words = set(stopwords.words('english'))

    # Loại bỏ các từ dừng
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Ghép lại các từ thành câu mới
    filtered_sentence = ' '.join(filtered_words)

    return filtered_sentence

# df_rating['reviewText'] = df_rating['reviewText'].apply(remove_stopwords)

def get_sentiment_words(text):
    # Khởi tạo SentimentIntensityAnalyzer
    sia = SentimentIntensityAnalyzer()

    # Tách câu thành các từ
    words = nltk.word_tokenize(text)

    # Lấy danh sách các từ ngừng (stop words)
    stop_words = set(stopwords.words('english'))

    # Danh sách các từ có cảm xúc
    sentiment_words = []

    # Kiểm tra từng từ trong câu
    for word in words:
        print(word, " - ", sia.polarity_scores(word)['compound'])
        # Nếu từ không phải là từ ngừng và có điểm tính cảm từ VADER lớn hơn 0
        # if word.lower() not in stop_words and sia.polarity_scores(word)['compound'] > 0:
        if word.lower() not in stop_words and sia.polarity_scores(word)['compound'] != 0:
            sentiment_words.append(word)

    return sentiment_words


def sentiment_analysis_flair(sentence):
    # Kiểm tra xem câu có rỗng không
    if not sentence.strip():
        return 0.0
    # Load pretrained sentiment model
    classifier = TextClassifier.load('en-sentiment')

    # Create a Flair Sentence
    flair_sentence = Sentence(sentence)

    # Predict sentiment
    classifier.predict(flair_sentence)
    # print(flair_sentence)
    # Get sentiment value (POSITIVE/NEGATIVE) and score
    sentiment_value = flair_sentence.labels[0].value
    sentiment_score = flair_sentence.labels[0].score
    score = 0.0
    bias = 1.0
    for keyword in keywords:
        if keyword.lower() in sentence.lower():
            bias = 1.75
    
    if(sentiment_value == 'POSITIVE'):
        score = sentiment_score * bias
    else:
        score = -sentiment_score * bias

    return score

#Define the text to analyze

def sentiment_analysis(text, model, tokenizer):
    # Tokenize the text
    # print(text)
    if not text:
        return -1
    text = text[:512]
    inputs = tokenizer(text, return_tensors='pt')

    # Perform sentiment analysis
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted probabilities for each class
    probs = softmax(outputs.logits, dim=1).squeeze()
    # print(probs)
    # Convert the probabilities to human-readable labels
    labels = [1, 2, 3, 4, 5]  # For this model, it predicts star ratings
    
    predicted_label = labels[torch.argmax(probs).item()]
    # print(predicted_label)
    return predicted_label

def check_string(input_value):
    # Kiểm tra xem giá trị có phải là chuỗi hay không
    if isinstance(input_value, str):
        # Nếu là chuỗi, trả về giá trị không thay đổi
        return input_value
    else:
        # Nếu không phải là chuỗi, trả về chuỗi rỗng
        return ''

def seperate_sentences(text):
    text = check_string(text)
    doc = nlp(text)
    # Khởi tạo một danh sách để lưu trữ các câu
    sentences = []
    current_sentence = ""
    # Duyệt qua mỗi từ trong văn bản
    for token in doc:
        # Nếu từ hiện tại không phải là dấu kết thúc câu, thêm từ vào câu hiện tại
        
        if token.is_sent_start:
            # Nếu có câu hiện tại, thêm câu vào danh sách câu
            if current_sentence:
                sentences.append(current_sentence.strip())
            # Bắt đầu một câu mới
            current_sentence = token.text
        elif token.text in ["but", "and", "so", "althought", "however", "because", "therefore", "meanwhile", "moreover", "nevertheless", "nonetheless", "otherwise", "thus", "yet"]:
            # print("that but:", current_sentence)
            if (brand.lower() in current_sentence.lower() for brand in brands):
                # print("need continue: ", current_sentence)
                current_sentence += " " + token.text
                continue
            elif current_sentence:
                sentences.append(current_sentence.strip())
                current_sentence = token.text
        else:
            # Thêm từ vào câu hiện tại
            current_sentence += " " + token.text
    # Thêm câu cuối cùng vào danh sách câu
    if current_sentence:
        sentences.append(current_sentence.strip())
    return sentences

def split_sentences_and_filter_sentiment(text):
    # Initialize SentimentIntensityAnalyzer
    sia = SentimentIntensityAnalyzer()

    # Tách đoạn văn thành các câu nhỏ
    # sentences = nltk.sent_tokenize(text)
    sentences = seperate_sentences(text)
    if len(sentences) == 1 and not re.search(r'[.!?]', text):
        sentences = [text]
    # print(sentences)
    # Lọc các câu không chứa bất kỳ từ miêu tả cảm xúc nào
    filtered_sentences = []
    for sentence in sentences:
        # print("---- sentence ------")
        if not sentence.strip():
            continue
        sentence = sentence.strip()
        words = nltk.word_tokenize(sentence)
        
        # for word in words:
        #     if(sia.polarity_scores(word)['compound'] != 0 ):
        #         print(word, ":", sia.polarity_scores(word)['compound'])

        contains_sentiment = (any(sia.polarity_scores(word)['compound'] != 0 for word in words) or 
                              any(word.lower() in pos_words or word.lower() in neg_words for word in words))
        for word in words:
            if(word.lower() in brands):
                contains_sentiment = False
        # Flair
        
        sentence_flair = Sentence(sentence)
        tagger.predict(sentence_flair)
        # print(sentence_flair.tokens)
        is_a = False
        for token in sentence_flair.tokens:
            for label in token.get_labels('pos'):
                if label.value in a:
                    # print(token, label.value)
                    is_a = True
                    break
        for word in words:
            if(word.lower() in brands):
                contains_sentiment = False
                is_a = False
        #--------------
        if contains_sentiment:
            filtered_sentences.append(sentence)
        elif is_a:
            filtered_sentences.append(sentence)
        else:
            print("remove: ", sentence)
            # print(evaluate_sentiment_flair(sentence))
        # print("---- end sentence ------")
    # Thêm câu vào danh sách nếu không có câu nào chứa từ miêu tả cảm xúc
    if not filtered_sentences:
        filtered_sentences = [""]
    # print(filtered_sentences)
    return filtered_sentences

[nltk_data] Downloading package punkt to /home/triet/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/triet/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/triet/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation', 'accolade', 'accolades', 'accommodative', 'accomodative', 'accomplish', 'accomplished', 'accomplishment', 'accomplishments', 'accurate', 'accurately', 'achievable', 'achievement', 'achievements', 'achievible', 'acumen', 'adaptable', 'adaptive', 'adequate', 'adjustable', 'admirable', 'admirably', 'admiration', 'admire', 'admirer', 'admiring', 'admiringly', 'adorable', 'adore', 'adored', 'adorer', 'adoring', 'adoringly', 'adroit', 'adroitly', 'adulate', 'adulation', 'adulatory', 'advanced', 'advantage', 'advantageous', 'advantageously', 'advantages', 'adventuresome', 'adventurous', 'advocate', 'advocated', 'advocates', 'affability', 'affable', 'affably', 'affectation', 'affection', 'affectionate', 'affinity', 'affirm', 'affirmation', 'affirmative', 'affluence', 'affluent', 'afford', 'affordable', 'affordably', 'afordable', 'agile', 'agilely', 'agility', 'agreeable', 'ag

In [151]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax
import torch

# Load pre-trained model and tokenizer for Amazon reviews
model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'  # Example model, you can choose other pre-trained models
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

In [152]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

predictRating = []
text = "I liked this song by the frames, so when I heard this version that is more on the acoustic side, ithought sounds O.K.! so I downloaded it, the main part of some is still in tact, the main difference is the girl and piano!!!!"
# text = remove_stopwords(text)


sentences = split_sentences_and_filter_sentiment(text)
total_sentiment = 0
for sentence in sentences:
    # sentence = remove_stopwords(sentence)
    score = sentiment_analysis_flair(sentence)
    print(sentence, score)
    total_sentiment += score
    # print(total_sentiment)
print(total_sentiment)
if total_sentiment < -0.2:
    predictRating.append(-1)
    print("Result: -1")
else:
    predictRating.append(1)
    print("Result: 1")

I liked this song by the frames , so when I heard this version that is more on the acoustic side , ithought sounds O.K. ! -0.7798442244529724
so I downloaded it , the main part of some is still in tact , the main difference is the girl and piano ! ! ! ! -0.892508327960968
-1.6723525524139404
Result: -1


In [154]:
import tqdm
import os


predictRating = []
filtered_sentences = []
def process_and_save(df_rating, output_file):

    output_df = pd.DataFrame(columns=['reviewerID', 'asin', 'predictRating', 'T_v', 'overall', 'reviewText', 'filteredReviewText'])
    score = 0
    # Đoạn mã dùng tqdm để hiển thị tiến trình %
    for i, row in tqdm.tqdm(df_rating.iterrows(), desc="Processing", total=len(df_rating)):
        sentences = split_sentences_and_filter_sentiment(row['reviewText'])
        filtered_sentences.append(" ".join(sentences))
        total_sentiment = sum(sentiment_analysis_flair(sentence) for sentence in sentences)
        if total_sentiment > 0.0:
            score = 1
        else:
            score = -1
        predictRating.append(score)
        # Lưu vào DataFrame và ghi vào file CSV sau mỗi 10 record
        output_df.loc[len(output_df)] = { 'reviewerID': row['reviewerID'],
                                          'asin': row['asin'],
                                          'predictRating': score,
                                          'T_v': row['T_v'],
                                          'overall': row['overall'],
                                          'reviewText': row['reviewText'],
                                          'filteredReviewText': " ".join(sentences) }
        
        if (i + 1) % 5 == 0:
            output_df.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)
            output_df = pd.DataFrame(columns=['reviewerID', 'asin', 'predictRating', 'T_v', 'overall', 'reviewText', 'filteredReviewText'])
    
    # Ghi phần còn lại của DataFrame vào file CSV
    if not output_df.empty:
        output_df.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)


# Sử dụng hàm
process_and_save(df_rating, extracted_data)


Processing:   0%|          | 1/10000 [00:03<11:00:19,  3.96s/it]

remove:  bien


Processing:   0%|          | 4/10000 [00:16<12:14:03,  4.41s/it]

remove:  What can I say ?


Processing:   0%|          | 9/10000 [00:45<19:42:22,  7.10s/it]

remove:  I had Lifesong


Processing:   0%|          | 11/10000 [01:24<37:58:33, 13.69s/it]

In [133]:
output = pd.read_csv('./DeepCGSR_triet/output_DigitalMusic_02.csv')
output_df = pd.DataFrame(output)
print("Length Original Dataset: ", len(output_df))
outliner = pd.DataFrame(columns=['reviewerID', 'asin', 'predictRating', 'T_v', 'overall', 'reviewText', 'filteredReviewText'])
filteredRecords = pd.DataFrame(columns=['reviewerID', 'asin', 'predictRating', 'T_v', 'overall', 'reviewText', 'filteredReviewText'])
for i, row in output_df.iterrows():
    if row['predictRating'] != row['T_v']:
        outliner.loc[len(outliner)] = row
    else:
        filteredRecords.loc[len(filteredRecords)] = row
        # print('------------------------')
outliner.to_csv('./DeepCGSR_triet/outliner_DM_02.csv', index=False)
filteredRecords.to_csv('./DeepCGSR_triet/filteredRecords_DM_02.csv', index=False)

print("Length Outliner Dataset: ", len(outliner))
print("Length Filered Dataset: ", len(filteredRecords))

Length Original Dataset:  10000
Length Outliner Dataset:  798
Length Filered Dataset:  9202


In [134]:
output = pd.read_csv('./DeepCGSR_triet/output_DigitalMusic_02.csv')
output_df = pd.DataFrame(output)
print("Length Original Dataset: ", len(output_df))
outliner = pd.DataFrame(columns=['reviewerID', 'asin', 'predictRating', 'T_v', 'overall', 'reviewText', 'filteredReviewText'])
filteredRecords = pd.DataFrame(columns=['reviewerID', 'asin', 'predictRating', 'T_v', 'overall', 'reviewText', 'filteredReviewText'])
for i, row in output_df.iterrows():
    if row['predictRating'] != row['T_v']:
        outliner.loc[len(outliner)] = row
    else:
        filteredRecords.loc[len(filteredRecords)] = row
        # print('------------------------')
outliner.to_csv('./DeepCGSR_triet/Results/Digital_Music_5/VastOutput/outliner_DM_02.csv', index=False)
filteredRecords.to_csv('./DeepCGSR_triet/Results/Digital_Music_5/VastOutput/filteredRecords_DM_00.csv', index=False)

print("Length Outliner Dataset: ", len(outliner))
print("Length Filered Dataset: ", len(filteredRecords))

Dataset:  10000


Processing:   1%|          | 58/9202 [00:00<00:38, 235.61it/s]/tmp/ipykernel_1085762/1791447021.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_data.loc[len(result_data)] = row
Processing:  22%|██▏       | 2069/9202 [00:49<02:50, 41.94it/s]


KeyboardInterrupt: 